In [23]:
from ase.calculators.castep import Castep
from ase.visualize import view
import ase
import os
import numpy as np
import warnings
warnings.simplefilter('ignore')
from glob import glob

In [20]:
def get_MP_grid(atoms_in, kspacing: float = 0.01):
    A = np.array(atoms_in.cell)
    x= A[0,:] ; y= A[1,:] ;z= A[2,:]
    V = atoms_in.get_volume()
    ix = (np.pi/V) *np.cross(y, z)
    iy = (np.pi/V) *np.cross(z, x)
    iz = (np.pi/V) *np.cross(x, y)
    mx, my, mz =int(np.round((1/3)*np.linalg.norm(ix)/kspacing) -1), \
            int(np.round((1/3)*np.linalg.norm(iy)/kspacing) -1), \
            int(np.round((1/3)*np.linalg.norm(iz)/kspacing)-1)
    return [mx, my, mz]

In [29]:
files = ['BeC_tube+wire.xyz',  'Be_nanowire.xyz',  'C_nanotube.xyz']
atoms = ase.io.read(f'../unopt_xyzs/{files[0]}')
view(atoms*[1,1,1], viewer='nglview')

In [21]:
param_string = "TASK: SinglePoint \n" \
"CUT_OFF_ENERGY: 1000 \n" \
"DATA_DISTRIBUTION: kpoint \n" \
"OPT_STRATEGY: speed \n" \
"XC_FUNCTIONAL: PBE \n" \
"GRID_SCALE: 2 \n" \
"FINE_GRID_SCALE: 2.0 \n" \
"FIX_OCCUPANCY: false \n" \
"METALS_METHOD: dm \n" \
"MIXING_SCHEME: pulay \n" \
"NEXTRA_BANDS: 35 \n" \
"SMEARING_WIDTH: 0.05 ev \n" \
"SPIN_POLARISED: false \n" \
"CALCULATE_STRESS : TRUE \n" \
" \n" \
"%BLOCK devel_code \n" \
"PARALLEL: kpoint=64 : ENDPARALLEL \n" \
"%ENDBLOCK devel_code \n"

calculator= Castep()
calculator.cell.kpoint_mp_grid =  '1 1 500'
calculator.cell.fix_all_cell = 'true'

In [22]:

atoms.set_calculator(calculator)
ase.io.write('convergence_study/BeC_tube+wire.cell', atoms , format='castep-cell')
f = open('convergence_study/BeC_tube+wire.param', "w")
f.write(param_string)
f.close()
